## Ucitavanje baze

In [1]:
from funkcije import *

In [2]:
baza_location = biraj_bazu()

FileChooser(path='/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/baza/emodb/wav', filen…

In [3]:
baza = ucitaj_bazu(baza_location)

greska sa citanjem fajla:/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/baza/emodb/wav/newbaza


/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/funkcije.py:123: UserWarning: PySoundFile failed. Trying audioread instead.
  data, samplerate = librosa.load(fc.selected + filename)
/home/zarko/.local/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [4]:
#liste ficera 
hurstovi = [] #+
duzine = [] #+
odnostig = [] #+
najduzaprica = [] #+

mfccovi = []

zcrovi = [] #lokalni
pitchovi = []

zcr_prosek = [] #globalni #+
zcr_min = []
zcr_max = []
zcr_med = []
zcr_stdev = []

pitch_prosek = [] #globalni 
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

energije_short_term = []
energije_short_term_silence = []

frame_lenght = 480 
hop_lenght = frame_lenght//2


frame_length_tisina = 100
hop_length_tisina = frame_length_tisina//2
granica = 0.01

---
## hurst 

In [21]:

for sample in baza:
        h, c, val = compute_Hc(sample.data)
        hurstovi.append(h)

### duzine

In [22]:


for sample in baza:

     duzina = len(sample.data) * 1/sample.samplerate
        
     duzine.append(duzina) 

In [23]:
### odnos govor prema tisina, max govor 

duzinatisine = []

#  frejmovano


for sample in baza:
      
     duzina = len(sample.data) * 1/sample.samplerate
     duzinatisinepriv = 0
     ptisina = 0
     trenprica = 0
     maxprica = 0
     for i in range (0, len(sample.data), hop_length_tisina):
          engparceta = sum(abs(sample.data[i:i+frame_length_tisina]**2))
          if engparceta < granica:
               trenprica = 0
               if ptisina == 0:
                    duzinatisinepriv = duzinatisinepriv + frame_length_tisina * 1/sample.samplerate
               else :
                    duzinatisinepriv = duzinatisinepriv + hop_length_tisina * 1/sample.samplerate
               ptisina = 1
          else: 
               ptisina = 0
               if trenprica == 0:
                    trenprica = frame_length_tisina * 1/sample.samplerate
               else :
                    trenprica = trenprica + hop_length_tisina * 1/sample.samplerate
               if trenprica > maxprica: maxprica = trenprica
     duzinatisine.append(duzinatisinepriv)
     najduzaprica.append(maxprica)

for i in range (0, len(duzine)):
     if duzinatisine[i] > 0:
          odnostig.append((duzine[i]-duzinatisine[i])/duzinatisine[i])
     else: odnostig.append(1)      

## MFCC
---

In [21]:
# data # citav audio fajl u nizu
# parcici[0] # prvi segment tog audio fajla(mozda nije prvi zbog tisine zapravo) --- NE
# prvi_seg # prvi segment koji nije tisina(trenutno su ostali pobrisani ovde)

mfccovi_array = []

for sample in baza:
    duzina = len(sample.data) * 1/sample.samplerate
            
    matrica = librosa.feature.mfcc (y = sample.data, sr = sample.samplerate)
    mfccovi.append(matrica) 
    niz = matrica.flatten()
    mfccovi_array.append(niz)


In [22]:
print(len(mfccovi_array))

535


In [15]:
print()


#for i in range (0, len(mfccovi)):
 #   print(mfccovi[i])
  #  print(mfccovi_array[i])

[[-573.01196   -582.12134   -586.8176    ... -492.71088   -531.3699
  -549.42737  ]
 [  40.86902     42.113426    34.175797  ...   80.37989     71.04459
    57.802822 ]
 [  22.214771    30.598276    32.493942  ...   -9.611155    10.583001
    20.991535 ]
 ...
 [   3.1278324    2.2764015    3.3137739 ...    5.9192276    4.657543
     2.0677433]
 [   4.8470674    2.6750722    2.5776076 ...   13.502996    13.43597
     7.1793947]
 [   4.293572     2.4341094    1.7580782 ...   -3.2146802   -3.3307939
     1.3146594]]
[-573.01196   -582.12134   -586.8176    ...   -3.2146802   -3.3307939
    1.3146594]
[[-544.9087     -524.06104    -542.695      ... -547.89197
  -538.6564     -559.7256    ]
 [  61.12242      83.625946     61.623817   ...   52.25776
    60.49017      40.590576  ]
 [  44.84446      53.43182      42.08766    ...   28.133833
    26.76817      27.031889  ]
 ...
 [   2.7556808     2.0682392     4.7842226  ...    1.3065519
     2.94589       3.7717164 ]
 [   0.8683192     1.0142984

IndexError: list index out of range

## ZCR
---

In [24]:
## lokalni 


for sample in baza:
        #pitch = librosa.feature.zero_crossing_rate(data, frame_length = frame_length, hop_length = hop_length)
        zcr = librosa.feature.zero_crossing_rate(sample.data, frame_length = frame_lenght, hop_length = hop_lenght)
        #print(pitch)
        #print(zcr[0])         
        #plt.plot(zcr)      #vraca matricu iz nekog razloga, ali je sav sadrzaj u zcr[0] tkd cu samo to da uzimam
        zcrovi.append(zcr[0])

In [25]:
## globalni 

for zcr in zcrovi:
    zcr_prosek.append(np.mean(zcr))
    zcr_min.append(np.min(zcr))
    zcr_max.append(np.max(zcr))
    zcr_med.append(np.median(zcr))
    zcr_stdev.append(np.std(zcr))
    #print (prosek, min, max, med, stdev)
    #break

## F0 --- pitch
---

In [26]:
#lokalni

for sample in baza:

    f0, voiced_flag, voiced_probs = librosa.pyin(sample.data,

                                                fmin=librosa.note_to_hz('C2'),

                                                fmax=librosa.note_to_hz('C7'), sr=sample.samplerate) #, frame_length=frame_len, hop_length=hop_len)
    pitchovi.append(f0)

In [ ]:
## globalni 

pitch_prosek = []
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

for pitch_nn in pitchovi:
    pitch = pitch_nn[pitch_nn > -1]
    pitch_prosek.append(np.mean(pitch))
    pitch_min.append(np.min(pitch))
    pitch_max.append(np.max(pitch))
    pitch_med.append(np.median(pitch))
    pitch_stdev.append(np.std(pitch))
    #print (prosek, min, max, med, stdev)
    #break

## energija
---

In [12]:
# energija

energije_ukupne = []
energije_short_term = []
energije_short_term_silence = []


# short term energija, frejmovano
frame_length = frame_length_tisina # videcemo ali ovo ce ukloniti vise tisine tkd trebalo bi da utice 
hop_length = frame_length//2


for sample in baza:

    energy = sum(abs(sample.data**2))  ## ukupna energija fajla
    energije_ukupne.append(energy)

    energy_arr = np.array([
        sum(abs( sample.data[i:i+frame_length]**2))
        for i in range(0, len(sample.data), hop_length)])

    energije_short_term_silence.append(energy_arr)
    
for energija_kratko in energije_short_term_silence:
    energije_short_term.append(energija_kratko[energija_kratko > granica])

KeyboardInterrupt: 

In [ ]:
plt.plot(energije_short_term_silence[0])
plt.plot(energije_short_term[0])


---
## normalizacija

In [9]:

hurstovi_norm = preprocessing.normalize([np.asarray(hurstovi)])
duzine_norm = preprocessing.normalize([np.asarray(duzine)])
odnostig_norm = preprocessing.normalize([np.asarray(odnostig)])
najduzaprica_norm = preprocessing.normalize([np.asarray(najduzaprica)])

zcr_prosek_norm = preprocessing.normalize([np.asarray(zcr_prosek)])
zcr_min_norm = preprocessing.normalize([np.asarray(zcr_min)])
zcr_max_norm = preprocessing.normalize([np.asarray(zcr_max)])
zcr_med_norm = preprocessing.normalize([np.asarray(zcr_med)])
zcr_stdev_norm = preprocessing.normalize([np.asarray(zcr_stdev)])

pitch_prosek_norm = preprocessing.normalize([np.asarray(pitch_prosek)])
pitch_min_norm = preprocessing.normalize([np.asarray(pitch_min)])
pitch_max_norm = preprocessing.normalize([np.asarray(pitch_max)])
pitch_med_norm = preprocessing.normalize([np.asarray(pitch_med)])
pitch_stdev_norm = preprocessing.normalize([np.asarray(pitch_stdev)])


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required by the normalize function.

---
## ucitavanje ficera u fajlove 

### ficeri_glb

In [10]:


for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "w")
    f.close()

hurstovi;
duzine;
odnostig;
najduzaprica;
zcr_prosek;
zcr_min;
zcr_max;
zcr_med;
zcr_stdev;
pitch_prosek;
pitch_min;
pitch_max;
pitch_med;
pitch_stdev;

filename, govornik, recenica, emocija, emo, pol, godiste

In [11]:

i = 0
for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "a")

    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")


    f.write(str(hurstovi_norm[i]) + ",")

    f.write(str(duzine_norm[i]) + ",")
    f.write(str(odnostig_norm[i]) + ",")
    f.write(str(najduzaprica_norm[i]) + ",")

    f.write(str(zcr_prosek_norm[i]) + ",")
    f.write(str(zcr_min_norm[i]) + ",")
    f.write(str(zcr_max_norm[i]) + ",")
    f.write(str(zcr_med_norm[i]) + ",")
    f.write(str(zcr_stdev_norm[i]) + ",")

    f.write(str(pitch_prosek_norm[i]) + ",")
    f.write(str(pitch_min_norm[i]) + ",")
    f.write(str(pitch_max_norm[i]) + ",")
    f.write(str(pitch_med_norm[i]) + ",")
    f.write(str(pitch_stdev_norm[i]) + ",")
    
    f.close()
    i = i+1

NameError: name 'hurstovi_norm' is not defined

### ficeri_glb_norm

In [12]:


for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb_norm/" + name + ".txt", "w")
    f.close()

In [13]:

i = 0
for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "a")

    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")


    f.write(str(hurstovi[i]) + ",")

    f.write(str(duzine[i]) + ",")
    f.write(str(odnostig[i]) + ",")
    f.write(str(najduzaprica[i]) + ",")

    f.write(str(zcr_prosek[i]) + ",")
    f.write(str(zcr_min[i]) + ",")
    f.write(str(zcr_max[i]) + ",")
    f.write(str(zcr_med[i]) + ",")
    f.write(str(zcr_stdev[i]) + ",")

    f.write(str(pitch_prosek[i]) + ",")
    f.write(str(pitch_min[i]) + ",")
    f.write(str(pitch_max[i]) + ",")
    f.write(str(pitch_med[i]) + ",")
    f.write(str(pitch_stdev[i]) + ",")
    
    f.close()
    i = i+1

IndexError: list index out of range

### ficeri_mfcc

In [15]:

for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_mfcc/" + name + ".txt", "w")
    f.close()

In [47]:
i = 0

for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_mfcc/" + name + ".txt", "w")
    velicina = str(mfccovi[i].shape)[1:3] + "," + str(mfccovi[i].shape)[5:7]
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
    f.close()
    i=i+1

In [48]:
print(velicina)
#print(velicina)
#print(velicina[:1] + "," + velicina[1:] + ",")

20,10


In [ ]:
print()